## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
keras = tf.keras

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
def seq2seq_window_dataset(series, window_size, batch_size=32):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)
  

def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [4]:
path = r'C:\Users\adria\1Python\Meu_novo_projeto\WIN_N_1K\WIN_N_1k.csv'
raw = pd.read_csv(path, index_col=0, parse_dates=True)

In [5]:
df = pd.DataFrame()

In [6]:
df['close'] = (raw['high'] + raw['low']) / 2

In [7]:
media_movel = df['close'].rolling(window=2).mean()

In [8]:
df['media_movel'] = media_movel

In [9]:
df

,close,media_movel
time,,
2022-01-18 11:38:00,106667.5,NaN
2022-01-18 11:39:00,106745.0,106706.25
2022-01-18 11:40:00,106767.5,106756.25
2022-01-18 11:41:00,106787.5,106777.50
2022-01-18 11:42:00,106882.5,106835.00
...,...,...
2022-10-13 16:55:00,114917.5,114917.50
2022-10-13 16:56:00,114917.5,114917.50
2022-10-13 16:57:00,114917.5,114917.50


In [10]:
df['r'] = df['media_movel'].pct_change()

In [11]:
df = df.dropna()

series = df['r'].values

time = np.arange(len(series))

In [12]:
len(series)

99997

In [13]:
len(time)

99997

In [14]:
#################

#df = pd.DataFrame()
#df['close'] = (raw['high'] + raw['low']) / 2
#media_movel = df['close'].rolling(window=2).mean()
#df['media_movel'] = media_movel

#df['r'] = df['close'].pct_change()

#df = df.dropna()

#series = df['r'].values

#time = np.arange(len(series))

In [15]:
split_time = 70000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [16]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [17]:
window_size = 64

In [18]:
train_set = TimeseriesGenerator(x_train, x_train, length=window_size, batch_size=512)

## WaveNet

In [24]:
%%time
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 64
train_set = TimeseriesGenerator(x_train, time_train, length=window_size, batch_size=512)
valid_set = TimeseriesGenerator(x_valid, time_valid, length=window_size, batch_size=512)

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for dilation_rate in (1, 2, 4, 8, 16, 32):
    model.add(
      keras.layers.Conv1D(filters=32,
                          kernel_size=2,
                          strides=1,
                          dilation_rate=dilation_rate,
                          padding="causal",
                          activation="relu")
    )
model.add(keras.layers.Conv1D(filters=1, kernel_size=1))
optimizer = keras.optimizers.Adam()
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])

model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50)
history = model.fit(train_set, epochs=500,
                    validation_data=valid_set,
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/500


ValueError: in user code:

    C:\Users\adria\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\adria\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\adria\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\adria\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\adria\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\adria\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:

In [ ]:
model = keras.models.load_model("my_checkpoint.h5")

In [ ]:
cnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
cnn_forecast = cnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, cnn_forecast)

In [ ]:
keras.metrics.mean_absolute_error(x_valid, cnn_forecast).numpy()

In [ ]:
keras.metrics.mean_squared_error(x_valid, cnn_forecast).numpy()

In [ ]:
df2 = pd.DataFrame()

df2['diff_moving_avg_plus_past'] = cnn_forecast
df2['x_valid'] = x_valid

ultimo_preco_fechamento = 113630.0

df2['preco_valid'] = (1 + df2['x_valid']).cumprod()
df2['preco_valid'] = ultimo_preco_fechamento * df2['preco_valid']

df2['desr_prev'] = (1 + df2['diff_moving_avg_plus_past']).cumprod()
df2['desr_prev'] = ultimo_preco_fechamento * df2['desr_prev']

In [ ]:
df2

In [ ]:
len(x_valid)

In [ ]:
len(x_train)

In [ ]:
df2['diff'] = df2['preco_valid']-df2['desr_prev']

In [ ]:
df2['diff'].mean()